In [ ]:
#from llama_index.core.node_parser import SentenceSplitter

In [ ]:
!python3 -m pip show llama-index

In [ ]:
%pip install llama-index-vector-stores-weaviate

In [ ]:
! pip install -U weaviate-client

! pip install youtube_transcript_api
! pip install llama_hub
! pip install llama_index

In [1]:
# running weaviate 
import os
import json
import weaviate

client = weaviate.Client("http://localhost:8080")


/Users/ama/mambaforge/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [ ]:
#from llama_index import SimpleDirectoryReader

#derm_doc = SimpleDirectoryReader('../data/external').load_data()

In [ ]:
import whisper

model = whisper.load_model("base")

playlist_link = "https://www.youtube.com/playlist?list=PLRcGnj2vLUduM8elSDybdOVv4NO-nAz_9"


# Create a Playlist object
playlist = Playlist(playlist_link)

all_docs = []


for video_url in playlist.video_urls[:50]:
    try:
        # Transcribe using Whisper API
        transcript = transcribe(url=video_url, language="en", task="transcribe")
        
        all_docs.extend(transcript)
    except Exception as e:
        print(f"An error occurred for video {video_url}: {e}")

In [ ]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

In [2]:
from pytube import Playlist
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
from youtube_transcript_api import TranscriptsDisabled


loader = YoutubeTranscriptReader()
playlist_link = "https://www.youtube.com/playlist?list=PLRcGnj2vLUduM8elSDybdOVv4NO-nAz_9"


playlist = Playlist(playlist_link)

def process_video(video_url):
    try:
        documents = loader.load_data(ytlinks=[video_url])

        return [doc for doc in documents]  # Assuming chunking is handled in loader 

    except TranscriptsDisabled:
        print(f"Transcripts are disabled for the video: {video_url}")
    except Exception as e:
        print(f"An error occurred for video {video_url}: {type(e).__name__}: {e}")
        return None  # Handle errors gracefully 
    
    

all_docs = []
for video_url in playlist.video_urls[:50]:
    processed_docs = process_video(video_url)
    if processed_docs:
        all_docs.extend(processed_docs)

Transcripts are disabled for the video: https://www.youtube.com/watch?v=YfZZRIjfp6E


In [3]:
output_folder = "../data/interim"
os.makedirs(output_folder, exist_ok=True)


#output file path
output_file_path = os.path.join(output_folder, "transcripts.json")

# list to store dictionaries for each document
json_data = []

# Iterate through all_docs and create a dictionary for each document
for doc in all_docs:
    doc_dict = {"Doc ID": doc.doc_id, "Text": doc.text}
    json_data.append(doc_dict)

# Save the list of dictionaries to a JSON file
with open(output_file_path, "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=2)

# Test for success
print(f"Transcripts saved to: {output_file_path}")

Transcripts saved to: ../data/interim/transcripts.json


In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
#from llama_index.core import Document, VectorStoreIndex

In [ ]:
# Assuming 'all_docs' is a list of documents with 'doc_id' and 'text' fields
documents = [{"doc_id": doc.doc_id, "text": doc.text} for doc in all_docs]

In [4]:


#documentts = [Document(text=t) for t in all_docs]
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
youTubeText = SimpleDirectoryReader('../data/interim/').load_data()


In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
## Weaviate
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader




In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

# bge embedding model
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

In [8]:
# ollama
Settings.llm = Ollama(model="mistral")

index = VectorStoreIndex.from_documents(
    youTubeText,
)

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("What is this video about?")
print(response)

ReadTimeout: timed out

In [ ]:

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
# construct vector store for embeddings
vector_store = WeaviateVectorStore(weaviate_client= client,index_name="Derm",  text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# set up the index
#index = VectorStoreIndex.from_documents(nodes, storage_context= storage_context)


In [ ]:
from llama_index.llms import Ollama
llm = Ollama(model="mistral")
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")


In [ ]:
# create the index; this will embed the documents and store them in the vector store
index = VectorStoreIndex.from_documents(all_docs,service_context=service_context,storage_context=storage_context)


In [ ]:
# query the index
query_engine = index.as_query_engine()
response = query_engine.query("What is this document about?. Give detail")
print(response)


In [ ]:
import weaviate
from llama_index.readers.weaviate.reader import WeaviateReader

reader = WeaviateReader("http://localhost:8080")

documents = reader.load_data(
     class_name="BlogPost", 
     properties=["content"], 
     separate_documents=True
)
